# Sparse Zonal Harmonic Factorization

In [ ]:
import numpy as np
import sph_harm as sh

lobe_dirs = np.array([
[0.0000, 0.0000],
[1.5708, 1.5708], [0.0000, 0.0000], [1.5708, 0.0000], 
[1.5708, 1.5708], [0.9553,-2.3562], [3.1416, 2.3562], [0.9553, 0.7854], [2.1863, 2.3562], 
[3.1416, 2.6180], [1.5708,-2.6180], [1.5708, 1.5708], [2.0344,-3.1416], [2.0344,-1.5708], [2.0344,-0.5236], [2.0344, 1.5708],
[1.5708, 0.7854], [1.1832, 0.0000], [1.5708,-3.1416], [1.1832, 0.7854], [3.1416, 0.0000], [1.5708, 1.5708], [1.5708, 0.3927], [2.2845,-1.5708], [0.8571,-3.1416],
[0.0000, 0.0000], [1.5708, 1.5708], [2.1863, 1.5708], [2.1863,-2.7489], [1.5708,-2.3562], [1.5708,-2.7489], [1.5708,-0.7854], [0.6997, 1.5708], [0.6997,-2.3562], [0.9553, 1.5708], [1.5708, 0.0000],
[1.5708, 0.7854], [1.0213,-2.6180], [2.1203,-1.5708], [1.5708,-1.5708], [3.1416, 1.5708], [1.5708, 0.5236], [2.1203, 1.5708], [1.8241, 1.5708], [0.5913,-0.3142], [1.8241,-1.5708], [2.1203,-3.1416], [1.5708, 0.3927], [2.3389,-1.5708],
[1.5708,-0.5236], [2.0719, 2.6180], [0.6928, 1.5708], [1.5708,-1.5708], [3.1416,-0.3927], [0.6928,-1.5708], [1.7989,-3.1416], [2.0053, 1.5708], [1.8518,-3.1416], [2.0053,-1.5708], [0.6928,-2.3562], [2.2040,-1.5708], [0.8755, 0.0000], [2.2040, 1.5708], [0.6928, 2.6180],
])


In [ ]:
def eq_D_l(l):
    return np.sqrt(4*np.pi/(2*l+1))

def eq_Y_l(l):
    start_idx = l**2
    end_idx = (l+1)**2
    dirs = lobe_dirs[start_idx : end_idx]

    cnt = 2*l + 1
    Y_l = np.zeros([cnt, cnt])
    for row in np.arange(0, cnt):
        w = dirs[row]
        for column in np.arange(0, cnt):
            m = column - l
            Y_l[row][column] = sh.sph_harm(m, l, w[0], w[1])
    return Y_l


def eq_A_hat(l):
    A_hat = np.linalg.inv(eq_Y_l(l))
    return A_hat


l = 1
A_hat = eq_A_hat(l)
print(A_hat/eq_D_l(l))